In [1]:
from pystoi import stoi
import librosa
import os
import numpy as np

In [2]:
# 加载音频文件并对齐
def load_wav(wavfile, sr=16000):
    """
    加载音频文件并返回音频信号
    """
    wav, _ = librosa.load(wavfile, sr=sr, mono=True)
    return wav

def align_signals(ref_signal, gen_signal):
    """
    对齐源语音和生成语音的长度，截短或填充到相同长度
    """
    min_len = min(len(ref_signal), len(gen_signal))
    ref_signal = ref_signal[:min_len]
    gen_signal = gen_signal[:min_len]
    return ref_signal, gen_signal

In [3]:
# 计算 STOI
def calculate_stoi(reference_signal, generated_signal, sr=16000):
    """
    计算 STOI 分数
    """
    # 对齐信号
    reference_signal, generated_signal = align_signals(reference_signal, generated_signal)
    # 计算 STOI 分数
    return stoi(reference_signal, generated_signal, sr, extended=False)


In [4]:
# 主流程函数
def process_stoi(mapping_file_path, original_base_path, generated_base_path):
    """
    从映射文件中提取源语音和转换语音，计算 STOI 分数
    """
    total_stoi = 0
    count = 0

    # 读取映射文件
    with open(mapping_file_path, 'r') as f:
        for line in f:
            parts = line.strip().split('|')

            # 检查是否有足够的字段
            if len(parts) != 3:
                print(f"Invalid line format: {line}")
                continue

            generated_wav, content_wav, _ = parts

            # 构建音频文件路径
            original_wav_path = os.path.join(original_base_path, content_wav)
            generated_wav_path = os.path.join(generated_base_path, generated_wav)

            # 检查文件是否存在
            if not os.path.exists(original_wav_path) or not os.path.exists(generated_wav_path):
                print(f"文件 {original_wav_path} 或 {generated_wav_path} 不存在，跳过此对。")
                continue

            # 加载源语音和生成语音
            reference_signal = load_wav(original_wav_path)
            generated_signal = load_wav(generated_wav_path)

            # 计算当前文件的 STOI 分数
            stoi_score = calculate_stoi(reference_signal, generated_signal)
            print(f"STOI Score for {generated_wav}: {stoi_score}")

            total_stoi += stoi_score
            count += 1

    # 计算平均 STOI 分数
    avg_stoi = total_stoi / count if count > 0 else 0
    print(f"\nAverage STOI Score: {avg_stoi}")

In [7]:
# 示例用法
mapping_file_path = '/home/sun/FreeVC/convert_A.txt'  # 转换文件路径
original_base_path = '/home/sun/FreeVC/outputs/wav_o'  # 原始音频目录
generated_base_path = '/home/sun/FreeVC/outputs/autovc_test'  # 生成音频目录

In [ ]:
process_stoi(mapping_file_path, original_base_path, generated_base_path)